In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import os
%matplotlib notebook  
%matplotlib inline  
#####################全局变量
All_stock_dict = {'Code':[],'Name':[],'Frequency':[],'DataDf':[]} # 储存所有股票的详细数据，StockData类中调用
stock_problem_data = {'Code':[],'Name':[]}
#####################
#####################
stock_num = 100           # 买入评分最高的前 stock_num 只股票
momentum_day = 29       # 最新动量参考最近 momentum_day 的
N = 18 # 计算最新斜率 slope，拟合度 r2 参考最近 N 天
M = 600 # 计算最新标准分 zscore，rsrs_score 参考最近 M 天
mean_day      = 20 # 计算结束 ma 收盘价，参考最近 mean_day
mean_diff_day = 3 # 计算初始 ma 收盘价，参考(mean_day + mean_diff_day)天前，窗口为 mean_diff_day 的一段时间
score_threshold = 0.7
#####################

In [2]:
# 此类用于处理从本地获得的单支股票数据，并放入全局变量All_stock_dict字典中，后面每调用一次类中的data_store函数，就存储相应股票信息
class StockData():   
    def __init__(self,url):
        self.data_df    = []
        self.url        = url
        self.f          = None
        self.read_line  = None
        self.line       = None
        self.stock_info = None
        self.stock_data = None
        self.stock_dict = {}
    def data_clean(self):
        try:
            with open(self.url,'r') as self.f:
                while True:
                    self.read_fline = self.f.readline()            # 按行读取
                    if not self.read_fline:
                        break
                    self.line = self.read_fline.strip()            # 去除空格和换行符
                    if not self.line:
                        continue
                    self.data_df.append(self.line.split('\t'))     # 按照“\t”分割字符串，并且放入data中    
        except:
            print('The data_clean fuction in StockData Class executes error') 
        self.data_df             =  pd.DataFrame(self.data_df)  
        self.stock_info          =  np.array(self.data_df.iloc[0])    # 取 dataframe 的第一行元素
        self.stock_info          =  self.stock_info[0].split()        # .split() 将字符串转化为列表
        self.stock_data          =  self.data_df.iloc[1:-1]           # 从dataframe取出真实价量数据
        self.stock_data          =  self.stock_data.iloc[::-1]        # dataframe 逆序排列。所得结果：最新数据行索引为【0】
        self.stock_data          =  self.stock_data.drop([1],axis=0)  # 删除第一行文字索引
        self.stock_data.index    =  np.arange(self.stock_data.shape[0])                     # 行索引命名
        self.stock_data.columns  =  ['Date','Open','High','Low','Close','Vol','Vocc']       # 列索引命名
#         self.stock_dict['Code']       =  self.stock_info[0]
#         self.stock_dict['Name']       =  self.stock_info[1]
#         self.stock_dict['Frequency']  =  self.stock_info[2]
#         self.stock_dict['Data']       =  self.stock_data
#         return self.stock_dict
        
    def data_store(self):
        dict={}
        self.data_clean()
        if not np.any(self.stock_data.isnull()) and len(self.stock_data) >= M:
            All_stock_dict['Code'].append(self.stock_info[0])           # 将本支股票的代码放入全局变量中
            All_stock_dict['Name'].append(self.stock_info[1])           # 将本支股票的名字放入全局变量中
            All_stock_dict['Frequency'].append(self.stock_info[-2])      # 将本支股票的看盘周期放入全局变量中
    #        All_stock_dict['DataDf'].append(self.stock_data)           # 将本支股票的历史数据放入全局变量中
            dict['Date']  =  np.array(self.stock_data['Date'])          # 将本支股票的历史数据放入全局变量中
            dict['Open']  =  np.array(self.stock_data['Open']).astype(float)          # 将本支股票的历史数据放入全局变量中
            dict['High']  =  np.array(self.stock_data['High']).astype(float)          # 将本支股票的历史数据放入全局变量中
            dict['Low']   =  np.array(self.stock_data['Low']).astype(float)           # 将本支股票的历史数据放入全局变量中
            dict['Close'] =  np.array(self.stock_data['Close']).astype(float)         # 将本支股票的历史数据放入全局变量中
            dict['Vol']   =  np.array(self.stock_data['Vol']).astype(float)           # 将本支股票的历史数据放入全局变量中
            dict['Vocc']  =  np.array(self.stock_data['Vocc']).astype(float)          # 将本支股票的历史数据放入全局变量中
            All_stock_dict['DataDf'].append(dict)
        else:
            stock_problem_data['Code'].append(self.stock_info[0])
            stock_problem_data['Name'].append(self.stock_info[1])
    # 根据函数参数选择指定数据长度的股票
#     def get_priceDict(self,stockCode,count,frequency,fields=['Data','Open','High','Low','Close','Vol','Vocc'],endDate=None):
#         self.dict={} 
#         self.stockCode=stockCode
#         self.count=count
#         self.frequency=frequency
#         self.fields=fields
#         self.endDate=endDate
#         for (self.C,self.N,self.F,self.D) in zip(All_stock_dict['Code'],All_stock_dict['Name'],All_stock_dict['Frequency'],All_stock_dict['DataDf']):
#             if self.C == self.stockCode and self.F == self.frequency:
#                 df = pd.DataFrame(self.D).iloc[:self.count,]               #  截取请求数量的dataframe的行       
#                 for i in range(len(self.fields)):                     # 遍历请求的索引
#                     dict[self.fields[i]] = np.array(df[self.fields[i]])    # 将获取的数列赋予字典等待返回
#                 break    
#         if self.dict == {}:
#             print("The get_priceList function of value is error")
#         else:
#             return self.dict

In [3]:
# 根据函数参数选择指定数据长度的股票
def get_priceDict(stockCode,count,frequency,fields=['Date','Open','High','Low','Close','Vol','Vocc'],endDate=None):
    dict={} 
    for (C,N,F,D) in zip(All_stock_dict['Code'],All_stock_dict['Name'],All_stock_dict['Frequency'],All_stock_dict['DataDf']):
        if C == stockCode and F == frequency:
            df = pd.DataFrame(D).iloc[:count,]               #  截取请求数量的dataframe的行       
            for i in range(len(fields)):                     # 遍历请求的索引
                dict[fields[i]] = np.array(df[fields[i]])    # 将获取的数列赋予字典等待返回
            break    
    if dict == {}:
        print('The get_priceList function of value is error{}'.format(stockCode))
    else:
        return dict
def get_stockName(stockCode): # 根据 股票代码 寻 股票名称
    a = None
    for (C,N) in zip(All_stock_dict['Code'],All_stock_dict['Name']):
        if C == stockCode:
            a = N
            break
    if a == None:
        print('Stock Code is error')
    else:
        return a
        

In [4]:
def get_ols(x, y):    # 对输入的自变量每日最低价 x(series) 和因变量每日最高价 y(series) 建立 OLS 回归模型,返回元组(截距,斜率,拟合度)
    slope, intercept = np.polyfit(x, y, 1)
    r2 = 1 - (sum((y - (slope * x + intercept))**2) / ((len(y) - 1) * np.var(y, ddof=1)))
    return (intercept, slope, r2) 
def initial_slope_series(ref_stock):
    data = get_priceDict(ref_stock,count = N + M,frequency="日线",fields=['High','Low'])
     # get_ols()函数返回的是元组(截距,斜率,拟合度)，这里调用后生成序列,索引【1】得到斜率序列
     # 采用移动窗口的方法计算前 M 天的回归斜率
    return [get_ols(data['Low'][::-1][i:i+N], data['High'][::-1][i:i+N])[1] for i in range(M)]
def get_zscore(slope_series):
    mean = np.mean(slope_series)
    std  = np.std(slope_series)             # 计算标准差
    return (slope_series[-1] - mean) / std # 标准分计算公式：Z = （X - Mean）/ 标准差
def get_timing_signal(ref_stock):
    # 计算 MA 信号，mean_day = 20，mean_diff_day =3
    close_data = get_priceDict(ref_stock,count = mean_day + mean_diff_day,frequency="日线",fields = ['Close']) 
    # 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1，23 天，要后 20 天
    today_MA   = close_data['Close'][::-1][mean_diff_day:].mean() 
    # 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0，23 天，要前 20 天
    before_MA  = close_data['Close'][::-1][:-mean_diff_day].mean()
    # 计算 rsrs 信号，N =18，M = 600
    high_low_data      = get_priceDict(ref_stock,count = N,frequency="日线",fields = ['High','Low'])
    intercept,slope,r2 = get_ols(high_low_data['Low'][::-1],high_low_data['High'][::-1])
    slope_series       = initial_slope_series(ref_stock)
    rsrs_score         = get_zscore(slope_series[-M:]) * r2
    # 综合判断所有信号
    if rsrs_score > score_threshold and today_MA > before_MA:
        return "BUY"
    elif rsrs_score < -score_threshold and today_MA < before_MA:
        return "SELL"
    else:
        return "KEEP"

def get_rank(stock_pool):
    score_list = []
    stock_pool_select = []
    for stock in stock_pool:
        data = get_priceDict(stock,count = momentum_day,frequency="日线",fields=['Close']) # momentum_day=29
        try:
            y = data['log'] = np.log(data['Close'][::-1]) # 求收盘价数组的Log值
            # 分析的数据个数（天）
            x = data['num'] = np.arange(len(data['log'])) # 生成从0开始排列的，与data【‘log’】数量相同的索引序列
            # 拟合 1 次多项式
            # y = kx + b, slope 为斜率 k，intercept 为截距 b
        except:
            print(stock)
        try:
            slope, intercept = np.polyfit(x, y, 1) # 拟合一次多项式，返回拟合直线的k、b值
        except:
            print('拟合错误')
        # (e ^ slope) ^ 250 - 1 ;annualized_returns 为年度回报率
        annualized_returns = math.pow(math.exp(slope), 250) - 1 # pow()方法返回x的y次幂的值
        # r_squared 是衡量线性回归效果的评价指标 R^2 =1 - sum（（yi - y）^2）/ sum((ymean - y)^2)
        r_squared = 1 - (sum((y - (slope * x + intercept))**2) / ((len(y) - 1) * np.var(y, ddof=1))) # np.var（）求方差
        score = annualized_returns * r_squared # 年化回报率*模型评价指标（负无穷--1】
        if np.isnan(score):
            stock_problem_data['Code'].append(stock)
            stock_problem_data['Name'].append(get_stockName(stock))
        else:
            score_list.append(score) # 评分列表
            stock_pool_select.append(stock)
    stock_dict = {'stock_pool_select':stock_pool_select, 'score_list':score_list} # 股票池与年化收益评价值绑定生成字典
    # 按照字典stock_dict中的score排名进行降序排列。stock_dict.items()为元组构成的列表 reverse = True 为降序排列
    sort_Df = pd.DataFrame(stock_dict).sort_values(by='score_list',ascending=False)
    print("#" * 30 + "候选 （打分）" + "#" * 30) # 打印##############################候选##############################
    for x in range(len(sort_Df)):
        stock_code    = np.array(sort_Df['stock_pool_select'])[x]
        stock_score   = np.array(sort_Df['score_list'])[x]
        stock_name    = get_stockName(stock_code)
        print('{}({}):{}'.format(stock_name,stock_code,stock_score)) # 打印所有股票的名字（股票的代号），打分
    print('#' * 73)
    code_list = []
    for i in range(len(stock_pool_select)):
        code_list.append(np.array(sort_Df['stock_pool_select'])[i])
    rank_stock = code_list[0:stock_num] # 取出stock_dict降序排序后前stock_num个股票的代号列表
    return rank_stock # 返回按照年化收益率排列最优的股票代号列表


In [5]:

# plt.scatter(x,y)  # 绘制散点图
# plt.xlabel('x')
# plt.ylabel('y')
# plt.plot(x,x*slope+intercept)

In [6]:
if __name__ == "__main__":
    list = os.listdir('D:/Program Files/通达信/T0002/export/')
    [StockData(url='D:/Program Files/通达信/T0002/export/' + i).data_store() for i in list]
    check_out_list = get_rank(All_stock_dict['Code'])
    for each_check_out in check_out_list:
        stock_name = get_stockName(each_check_out)
        stock_code = each_check_out
        print('今日自选股:{}({})'.format(stock_name, stock_code))
    #获取综合择时信号
    timing_signal = get_timing_signal('999999')
    print('今日择时信号:{}'.format(timing_signal))
    print('*' * 100)
    #print('访问数据出错的股票:{}({})'.format(stock_problem_data['Name'],stock_problem_data['Code']))

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'D:/Program Files/通达信/T0002/export/'